In [ ]:
# %pip install pymssql

In [57]:
import pymssql
import pandas as pd
import redis
import time
import datetime

In [58]:
REDIS_IP = '10.2.5.212'

In [59]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
            autocommit=True
        )

In [74]:
def read_sql(query):
    return pd.read_sql(query, con=ms_sql_con(), parse_dates=None)

In [109]:
def concatenate_linkedid_side(side, record_date, linkedid):
    query = "SELECT text from transcribations where "
    query += " side="+str(side)+" and "
    query += " record_date = '"+str(record_date)+"' and "
    query += " linkedid = '"+str(linkedid)+"';"
    text_df = read_sql(query)
    phrases_count = len(text_df)
    text_full = ', '.join([row.text for _id, row in text_df.iterrows()])
    return text_full, phrases_count

In [86]:
def summarize(text, phrases_count):
    if phrases_count<2 or len(text)<255:
        return text
    subscriber = redis.StrictRedis(host=REDIS_IP)
    publisher = redis.StrictRedis(host=REDIS_IP) 
    pub = publisher.pubsub()
    sub = subscriber.pubsub()
    sub.subscribe('summarus_client')
    # send
    publisher.publish("summarus_server", text)
    # receive
    while True:
        message = sub.get_message()
        if message and message['type']!='subscribe':
            return message['data'].decode("utf-8")
        time.sleep(1)

In [77]:
def sum_to_sql(linkedid, recor_date, side, text, phrases_count, text_length):
    current_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    query = "insert into summarization(linkedid, record_date, sum_date, side, text, phrases_count, text_length) "
    query += " values("
    query += "'"+str(linkedid)+"',"
    query += "'"+str(recor_date)+"',"
    query += "'"+str(current_date)+"',"
    query += str(side)+","
    query += "'"+str(text)+"',"
    query += "'"+str(phrases_count)+"',"
    query += "'"+str(text_length)+"'"
    query += ");"
    #df = read_sql(query)
    
    conn = ms_sql_con()  
    cursor = conn.cursor()
    cursor.execute(query)
    # conn.commit()

In [ ]:
# queue
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
queue = read_sql(query)
queue

In [ ]:
# transcribations
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
queue = read_sql(query)
queue

### queue linkedid list

In [ ]:
# queue
query = "SELECT distinct record_date, linkedid from queue;"
queue = read_sql(query)
len(queue)

### transcribations linkedid list

In [ ]:
# queue
query = "SELECT distinct top 10 record_date, linkedid from transcribations;"
df = read_sql(query)
df

In [ ]:
query = "SELECT distinct top 100 trans.record_date, trans.linkedid"
query += " from transcribations as trans"
query += " left join queue as queue"
query += " on trans.record_date=queue.record_date"
query += " and trans.linkedid=queue.linkedid"
query += " where "
query += " trans.record_date > '2021-06-04 00:00:00' and"
query += " isnull(queue.linkedid,'x')='x'"
query += " order by record_date desc;"
%time df = read_sql(query)
len(df)

# start

In [123]:
# obtain datetime limits
query = "select min(record_date) from queue;"
df = read_sql(query)
queue_first_record = str(df.iloc()[0][0])
query = "select max(record_date) from summarization;"
df = read_sql(query)
summarization_first_record = str(df.iloc()[0][0])

In [124]:
query = "SELECT distinct top 1000 record_date, linkedid"
query += " from transcribations"
query += " where "
query += " record_date < '"+queue_first_record+"' and"
#query += " record_date >= '"+summarization_first_record+"'"
#query += " not linkedid in (select distinct linkedid from queue) and"
query += " not linkedid in (select distinct linkedid from summarization)"
query += " order by record_date desc;"
%time df = read_sql(query)
len(df)

CPU times: user 4.79 ms, sys: 710 µs, total: 5.5 ms
Wall time: 46.5 s


6

In [125]:
#text_full

In [126]:
for _id, row in df.iterrows():
    
    for side in range(2):
        text_full, phrases_count = concatenate_linkedid_side(side, row.record_date, row.linkedid)
        text_short = summarize(text_full, phrases_count)
        sum_to_sql(row.linkedid, row.record_date, side, text_short, phrases_count, len(text_full))

In [128]:
len('Российский мастер по ремонту стиральных машин, с кем можно переговорить, пожаловался на то, что белье не попадает в барабан, барабан не крутится, не вращается, он перетащил машинку с одного места на второе, а она почему-то перестала стирать, да, да нет москва, а я давно как ты взял эту самую визитку у этого почтовом ящике и вот по ней и звоню айсберг вот визитка, да конечно номер триста шестнадцать серия, спасибо спасибо, да пожалуй, да.')

441

In [131]:
# select
query = "SELECT top 12 * from summarization order by sum_date desc;"
df = read_sql(query)
for _id, row in df.iterrows():
    #df.text.iloc()[3]
    print(row.sum_date, row.text)

2021-06-04 15:26:08 Позвонив в единую службу сервиса, оператор валерия сообщила, что в ближайшее время к ней приедет мастер, который запишет заявку на ремонт ноутбука в москве.
2021-06-04 15:26:03 Российский москвич, который по воскресеньям работает в московском районе Щербаковская, пожаловался на то, что у него не работает включатель телевизора, а у него красная галочка от пульта моргает.
2021-06-04 15:25:58 В василисе произошла авария с сплит-системой, в результате которой погибли три человека, в том числе и я.
2021-06-04 15:25:53 о обойдясь скажите пожалуйста а устанавливать систему, длина третий этаж сколько готовить, минутку, наверно мне надо сначала узнать сумму заказу, ну хорошо тогда я поняла, не надо спасти
2021-06-04 15:25:50 В общем, я не знаю, как вы, а я просто хочу рассказать вам о том, что у меня в квартире, что я хочу, что мне не нравится, что не нравится и что я не хочу, потому что у нас плохая вентиляция.
2021-06-04 15:25:44 Скажите, пожалуйста, почему у меня на кухне

In [73]:
row

linkedid                                         1622796230.235140
record_date                                    2021-06-04 11:43:52
sum_date                                       2021-06-04 14:35:00
side                                                          True
load_msk                                                      None
load_spb                                                      None
load_reg                                                      None
text             Westwood и его партнёры уже не первый год пров...
phrases_count                                                    2
text_length                                                    196
Name: 3, dtype: object

query = "SELECT column_name FROM information_schema.columns WHERE table_name='summarization';"
queue = read_sql(query)
queue

In [132]:
# select
query = "SELECT side, text from transcribations where "
query += " record_date = '2021-06-04 12:33:20' and linkedid = '1622799198.236534';"
df = read_sql(query)
', '.join([row.text for _id, row in df.iterrows()])

'здравствуйте скажите пожалуйста мастера можно вызвать по ремонту холодильников сна, москве, стянул, он бы уже давно да, двух, он там где морозильник не сама м-ра телка вот обычно помещаю стал работать как морозилку, да да да, о нет магнитики разброс, вверху справа но мне наверно старая ещё три семнадцать, восемь девятьсот девять, девятьсот восемьдесят один пятьдесят один, семьдесят, галина, улица онежская, дом тридцать пять, корпус два квартира двести пятьдесят семь, третий подъезд пятый этаж без лифта, водный стадион, да, сто пятьдесят девять сорок пять восемьдесят четыре, хорошо спасибо, и, кузбасс сервиса оператору, могу принять заявку в течение часа мастер перезвонит у него все уточнить и подберёте временно визит живете в москве и области, холодильник как называется, гарантия закончилась от производителя холодильник двухкамерные трёхкамерные что не работает, и, промерзание шкафа да да, вы нас стефан узнали где магнит визитка сервисная книжка, вверху справа три цифры какие стоят на

In [133]:
len('здравствуйте скажите пожалуйста мастера можно вызвать по ремонту холодильников сна, москве, стянул, он бы уже давно да, двух, он там где морозильник не сама м-ра телка вот обычно помещаю стал работать как морозилку, да да да, о нет магнитики разброс, вверху справа но мне наверно старая ещё три семнадцать, восемь девятьсот девять, девятьсот восемьдесят один пятьдесят один, семьдесят, галина, улица онежская, дом тридцать пять, корпус два квартира двести пятьдесят семь, третий подъезд пятый этаж без лифта, водный стадион, да, сто пятьдесят девять сорок пять восемьдесят четыре, хорошо спасибо, и, кузбасс сервиса оператору, могу принять заявку в течение часа мастер перезвонит у него все уточнить и подберёте временно визит живете в москве и области, холодильник как называется, гарантия закончилась от производителя холодильник двухкамерные трёхкамерные что не работает, и, промерзание шкафа да да, вы нас стефан узнали где магнит визитка сервисная книжка, вверху справа три цифры какие стоят на магнитных карт сможет отказать, на, спасибо мы указываем адрес телефон в течение час поступит звонок телефон вас скажите, зовут вас как, гален улиц какая у вас, номер дома, корпус есть, квартира, подъезд и этаж, метро какое-то ближайшее, давайте проверим а несколько тридцать пять корпус два квартира два пять семь подъезд третий этаж пятый телефон девятьсот девять девять восемь один пять один семьдесят северном, техника авто номер вашей заявки будет сто пятьдесят девять сорок пять восемьдесят четыре, сорок пять восемьдесят четыре ждите звонок пожалуйста, пожалуйста свидания')

1579

In [135]:
a = 'hello world'
a[:3]

'hel'

In [33]:
#df.text.iloc()[3]

'Germany. Москва, улица матроса железняка дом тридцать шесть квартира шестьдесят семь. подъезд четыре этаж два код домофона четыре пятнадцать. двенадцати до четырнадцати. досками.'

# DELETE

In [134]:
# delete
query = "DELETE from summarization;"
conn = ms_sql_con()  
cursor = conn.cursor()
cursor.execute(query)